In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import lightgbm as lgb
import catboost
from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)



from tqdm import tqdm
from itertools import product
from sklearn.model_selection import train_test_split
import seaborn as sns

import sys
import os
import gc
from glob import glob
import pickle
import json
import subprocess
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, train_test_split, RepeatedStratifiedKFold


In [ ]:
# !pip install --no-index --find-links ../input/pytorchset timm
# !pip install --no-index --find-links ../input/pytorchset pytorch-lightning

In [ ]:
# import torch
# import torch.optim as optim
# import torch.nn as nn
# import torch.nn.functional as F

# import torchvision.transforms as T
# import timm
# from sklearn.model_selection import StratifiedKFold
# from torchvision.io import read_image
# from torch.utils.data import DataLoader, Dataset

# import pytorch_lightning as pl
# from pytorch_lightning.utilities.seed import seed_everything
# from pytorch_lightning import callbacks
# from pytorch_lightning.callbacks.progress import ProgressBarBase
# from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# from pytorch_lightning.loggers import TensorBoardLogger
# from pytorch_lightning import LightningDataModule, LightningModule

# from sklearn.metrics import mean_squared_error
# from scipy.optimize import minimize

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# import lightgbm as lgb

# import cupy as cp # linear algebra
# import cudf as cd # data processing, CSV file I/O (e.g. pd.read_csv)

# from cuml.svm import SVR
# from cuml.decomposition import PCA

# from cuml.metrics import accuracy_score

In [ ]:
DF_CEILING_VALUE = 20000.0
SHOP_CEILING_VALUE = 999999.9
ITEM_CEILING_VALUE = 999999.9


In [ ]:
PATH = '../input/competitive-data-science-predict-future-sales/'

df = pd.read_csv(PATH + 'sales_train.csv')
df_test = pd.read_csv(PATH + 'test.csv')
sample = pd.read_csv(PATH + 'sample_submission.csv')
items = pd.read_csv(PATH + 'items.csv')
shops = pd.read_csv(PATH + 'shops.csv')
item_cats = pd.read_csv(PATH + 'item_categories.csv')

In [ ]:
data_files_names = ["df","df_test","sample","items","shops","item_cats"]
data_files = [df,df_test,sample,items,shops,item_cats]

Preprocessing each data

In [ ]:
df_test = df_test.drop('ID', axis=1)

In [ ]:
df

In [ ]:
df.loc[df['item_cnt_day'] < 0.0, 'item_cnt_day'] = 0.0

In [ ]:
df[df['item_cnt_day'] > 100.0]

In [ ]:
df.loc[df['item_cnt_day'] > 100.0, 'item_cnt_day'] = 1.0
len(df[df['item_cnt_day']>10.0])

In [ ]:
df[df['item_cnt_day'] > 10.0]

In [ ]:
df.loc[df['shop_id'] == 57, 'shop_id'] = 0
df.loc[df['shop_id'] == 58, 'shop_id'] = 1
df.loc[df['shop_id'] == 11, 'shop_id'] = 10
df.loc[df['shop_id'] == 40, 'shop_id'] = 39

df_test.loc[df_test['shop_id'] == 57, 'shop_id'] = 0
df_test.loc[df_test['shop_id'] == 58, 'shop_id'] = 1
df_test.loc[df_test['shop_id'] == 11, 'shop_id'] = 10
df_test.loc[df_test['shop_id'] == 40, 'shop_id'] = 39

In [ ]:
df['shop_item_id'] = df['shop_id'].astype('str').str.zfill(2) +  df['item_id'].astype('str').str.zfill(5)
df['item_category_id'] = pd.merge(df, items, on='item_id',how='left')['item_category_id']
df

In [ ]:
df = df.groupby(['date_block_num', 'shop_id','item_id','shop_item_id'], as_index=False
        ).agg({'item_cnt_day':'sum'}
        ).rename(columns={'item_cnt_day':'mon_shop_item_cnt'})
df

In [ ]:
# Add shop_item_id to df_test
df_test['shop_item_id'] = df_test['shop_id'].astype('str').str.zfill(2) + df_test['item_id'].astype('str').str.zfill(5)
df['item_category_id'] = pd.merge(df, items, on='item_id',how='left')['item_category_id']
df_test

In [ ]:
count = 0
df_ids = df['shop_item_id'].unique()
repeat_count = 0
for one_id in df_test['shop_item_id'].sample(1000):
    if one_id in df_ids:
        count += 1
    repeat_count += 1

    if repeat_count > 1000:
        break
print(count / repeat_count)

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(df['mon_shop_item_cnt'])

In [ ]:
df.loc[df['mon_shop_item_cnt'] > 100.0, 'mon_shop_item_cnt'] = 101.0

In [ ]:
print(len(df.loc[df['mon_shop_item_cnt'] > 100.0]), len(df.loc[df['mon_shop_item_cnt'] > 101.0]))

In [ ]:
# # left 214200
# transition = df_test
# for i in range(34):
#     transition = pd.merge(transition, df[df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i }), how='left')
# # transition = prepro_transition(transition, DF_CEILING_VALUE)
# transition = transition.fillna(0)
# transition

In [ ]:
# Outer -> 520915
transition = pd.DataFrame(np.unique(np.concatenate([df['shop_item_id'], df_test['shop_item_id']])), columns=['shop_item_id'])
for i in range(34):
    transition = pd.merge(transition, df[df['date_block_num']==i].drop(['date_block_num', 'shop_id', 'item_id', 'item_category_id'], axis=1).rename(columns={'mon_shop_item_cnt': i}), on='shop_item_id', how='left')
transition = transition.fillna(0)
transition

In [ ]:
plt.figure(figsize=(12,6))
plt.bar(transition.loc[:, 0:].columns,transition.loc[:, 0:].sum())

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(transition.loc[:, 0:].T.sum())

In [ ]:
# I think these data is too big but not invalid
transition[transition.loc[:, 0:].T.sum() >= 500.0]

In [ ]:
# transition_mean = transition.loc[:, 0:].mean().mean()
# transition_std = transition.loc[:, 0:].std().std()
transition_max = transition.loc[:, 0:].max().max()
transition_max

In [ ]:
std_transition = transition.copy()
std_transition.loc[:, 0:] = (std_transition.loc[:, 0:]) / transition_max
std_transition

In [ ]:
shops.loc[shops['shop_id'] == 57, 'shop_id'] = 0
shops.loc[shops['shop_id'] == 58, 'shop_id'] = 1
shops.loc[shops['shop_id'] == 11, 'shop_id'] = 10
shops.loc[shops['shop_id'] == 40, 'shop_id'] = 39
shops

In [ ]:
shop_df = df.groupby(['date_block_num', 'shop_id'], as_index=False
        ).agg({'mon_shop_item_cnt':'sum'}
        )
shop_transition = pd.DataFrame(shops['shop_id'].unique(), columns=['shop_id'])

for i in range(34):
    shop_transition = pd.merge(shop_transition, shop_df[shop_df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), on='shop_id', how='left')
shop_transition = shop_transition.fillna(0)

shop_transition_max = shop_transition.loc[:, 0:].max().max()
shop_transition.loc[:, 0:] = (shop_transition.loc[:, 0:]) / shop_transition_max

shop_transition

In [ ]:
shop_feature = transition.loc[:, ['shop_item_id']].copy()
shop_feature['shop_id'] = shop_feature['shop_item_id'].str[:2].astype(int)
shop_feature

In [ ]:
shop_feature = pd.merge(shop_feature, shop_transition, on='shop_id', how='left')
shop_feature = shop_feature.drop('shop_id', axis=1)
shop_feature

In [ ]:
items

In [ ]:
item_df = df.groupby(['date_block_num', 'item_id'], as_index=False
        ).agg({'mon_shop_item_cnt':'sum'})
item_transition = pd.DataFrame(items['item_id'].unique(), columns=['item_id'])

for i in range(34):
    item_transition = pd.merge(item_transition, item_df[item_df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), on='item_id', how='left')
item_transition = item_transition.fillna(0)

item_transition_max = item_transition.loc[:, 0:].max().max()
item_transition.loc[:, 0:] = (item_transition.loc[:, 0:]) / item_transition_max

item_transition

In [ ]:
item_feature = transition.loc[:, ['shop_item_id']].copy()
item_feature['item_id'] = item_feature['shop_item_id'].str[2:].astype(int)
item_feature = pd.merge(item_feature, item_transition, on='item_id', how='left')
item_feature = item_feature.drop('item_id', axis=1)
item_feature

In [ ]:
cats_df = df.groupby(['date_block_num', 'item_category_id'], as_index=False
        ).agg({'mon_shop_item_cnt':'sum'})
cats_transition = pd.DataFrame(cats_df['item_category_id'].unique(), columns=['item_category_id'])
for i in range(34):
    cats_transition = pd.merge(cats_transition, cats_df[cats_df['date_block_num']==i].drop('date_block_num', axis=1).rename(columns={'mon_shop_item_cnt': i}), on='item_category_id', how='left')
cats_transition = cats_transition.fillna(0)

cats_transition_max = cats_transition.loc[:, 0:].max().max()
cats_transition.loc[:, 0:] = (cats_transition.loc[:, 0:]) / cats_transition_max

cats_transition

In [ ]:
cats_feature = transition.loc[:, ['shop_item_id']].copy()
cats_feature['item_id'] = cats_feature['shop_item_id'].str[2:].astype(int)
cats_feature['item_category_id'] = pd.merge(cats_feature, items, on='item_id',how='left')['item_category_id']
cats_feature = pd.merge(cats_feature, cats_transition, on='item_category_id', how='left')
cats_feature = cats_feature.drop(['item_id','item_category_id'], axis=1)
cats_feature

In [ ]:
print(shop_feature.loc[:, 0:].mean().mean(), item_feature.loc[:, 0:].mean().mean())

In [ ]:
# Add noise
shop_feature.loc[:, 0:] += np.random.normal(0, shop_feature.loc[:, 0:].mean().mean() * 0.025, shop_feature.loc[:, 0:].shape)
item_feature.loc[:, 0:] += np.random.normal(0, item_feature.loc[:, 0:].mean().mean() * 0.025, item_feature.loc[:, 0:].shape)
cats_feature.loc[:, 0:] += np.random.normal(0, cats_feature.loc[:, 0:].mean().mean() * 0.025, cats_feature.loc[:, 0:].shape)

In [ ]:
shop_feature

In [ ]:
features = [std_transition, shop_feature, item_feature, cats_feature]
# features = [std_transition, item_feature]

I want to know submission data's charactaristic.
Thus, I submit some single value

### Single value scores
* 0.2 : 1.220329 and 1.206208.
* 0.3 : 1.217545 and 1.203262.
* 0.4 : 1.222959 and 1.208611.
* 0.5 : 1.23646
* 0.6 : 1.245039 and 1.235736.

In [ ]:
pre_ave = 0.3

In [ ]:
# Generate dataset
index = std_transition.index

train_index = []
val_index = []
test_index = []

train_index = index
# train_index, val_index = train_test_split(index, test_size=0.2)
# val_index, test_index = train_test_split(val_index, test_size=0.2)

In [ ]:
# print(len(std_transition),len(train_index), len(val_index))
print(len(std_transition),len(train_index), len(val_index), len(test_index) )

In [ ]:
std_transition.loc[:20000, 0:].T.sum().sort_values()

In [ ]:
std_transition.loc[18252]

In [ ]:
df_test

In [ ]:
sub_index = pd.DataFrame(std_transition.index, columns=['index'])
sub_index['shop_item_id'] = std_transition['shop_item_id']
sub_index = pd.merge(df_test, sub_index, on='shop_item_id',how='left')
print(sub_index.isna().sum())
sub_index = sub_index.fillna(18252)
sub_index = sub_index['index']
sub_index

In [ ]:
val_index = sub_index
val_index

In [ ]:
std_pre_ave = (pre_ave / transition_max)
# std_pre_ave = adj_y_val.mean()
print(std_pre_ave, pre_ave, transition_max)

In [ ]:
class WindowGenerator():
    def __init__(self, start_month, last_target_month, input_width, train_index=train_index, val_index=val_index, test_index=test_index, sub_index=sub_index, features=features):
        self.start_month = start_month
        self.last_target_month = last_target_month
        
        # Work out the window parameters.
        self.input_width = input_width
        # It includes y value
        self.total_width = input_width + 1
        
        # Store the raw data.
        self.train_index = train_index
        self.val_index = val_index
        self.test_index = test_index
        self.sub_index = sub_index
        
        # Fixed
        self.features = features
        self.features_number = len(features)
        
        self.repeat_number = self.last_target_month - self.start_month - self.input_width
    def __repr__(self):
        return '\n'.join([
            f'Start month: {self.start_month}',
            f'Input window size: {self.input_width}',
            f'last_target_month: {self.last_target_month}',
            
            f'repeat_number: {self.repeat_number}',
            
            f'features_number: {self.features_number}',
        ])
    
    def generate_window(self, features_lists, index_i, start_point):
        one_window = []
        for window_i in range(self.input_width):
            one_features = []
            for feature_i in range(self.features_number):
                # Array start zero, so I plus 1 to start point.
                one_features.append(features_lists[feature_i][index_i][start_point + 1 + window_i])
            one_window.append(one_features)

        return one_window
    
    def generate_list(self, index):
        x_data = []
        y_data = []
        
        index_number = len(index)
        features_lists = [feature.loc[index].drop('shop_item_id', axis=1).to_numpy().tolist() for feature in features]
        
        reduce = 0
        
        # (batch, time, features)
        for index_i in range(index_number):
            for time_i in range(self.repeat_number):
                start_point = self.start_month + time_i
                one_y = features_lists[0][index_i][start_point + self.total_width]
                
                if self.repeat_number > 2:
                    if one_y < std_pre_ave:
                        if np.random.rand() < 0.3:
                            continue
                
                one_window = self.generate_window(features_lists, index_i, start_point)
                
                x_data.append(one_window)
                y_data.append(one_y)

        return x_data, y_data
    def generate_sub_x(self):
        features_lists = [feature.loc[self.sub_index].drop('shop_item_id', axis=1).to_numpy().tolist() for feature in features]
        index_number = len(self.sub_index)
        
        # start_point + self.total_width = 34
        start_point = 34 - self.total_width
        
        sub_x = []
        # (batch, time, features)
        for index_i in range(index_number):
             sub_x.append(self.generate_window(features_lists, index_i, start_point))
                
        return sub_x
    
    def make_dataset(self, index):
        x_data, y_data = self.generate_list(index)
        return tf.Dataset.from_tensor_slices((x_data, y_data))

    def train_list(self):
        return self.generate_list(self.train_index)

    def val_list(self):
        return self.generate_list(self.val_index)

    def test_list(self):
        return self.generate_list(self.test_index)
    
    
    @property
    def train(self):
        return self.make_dataset(self.train_index)

    @property
    def val(self):
        return self.make_dataset(self.val_index)

    @property
    def test(self):
        return self.make_dataset(self.test_index)


In [ ]:
input_width = 29

train_window = WindowGenerator(
    start_month = 30 - (input_width + 1),
    last_target_month = 32,
    input_width = input_width,
)
train_window

In [ ]:
val_window = WindowGenerator(
    start_month = 33 - (input_width + 1),
    last_target_month = 33,
    input_width = input_width,
)
val_window

In [ ]:
%%time
x_train, y_train = train_window.train_list()

In [ ]:
%%time
x_val, y_val = val_window.val_list()

In [ ]:
adj_x_train = np.array(x_train)
adj_y_train = np.array(y_train)
adj_y_train = adj_y_train.clip(0, 20.0 / transition_max)
print(adj_y_train.mean(), adj_y_train.std(), adj_y_train.max())

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(np.sqrt(adj_y_train))

In [ ]:
# One time delete 0.01%
over_index = np.where(adj_y_train < std_pre_ave)[0]
print(over_index)
print(len(adj_y_train), len(over_index))
drop_rate = 0.035
drop_number = int(len(over_index) * drop_rate) if int(len(over_index) * drop_rate) > 1.0 else 1
print(drop_number)

adj_area = 0.0001

In [ ]:
%%time
for i in range(100):
    over_index = np.where(adj_y_train < std_pre_ave)[0]
    np.random.shuffle(over_index)
    drop_index = over_index[:drop_number]
    drop_number = int(drop_number * 0.95)
    
    adj_x_train = np.delete(adj_x_train, drop_index, 0)
    adj_y_train = np.delete(adj_y_train, drop_index)
    if adj_y_train.mean() + adj_area > std_pre_ave:
        break
print(f'Stop i:{i}')
print(adj_y_train.mean(), adj_y_train.std(), adj_y_train.max())

In [ ]:
adj_x_val = np.array(x_val)
adj_y_val = np.array(y_val)
adj_y_val = adj_y_val.clip(0, 20.0 / transition_max)
print(adj_y_val.mean(), adj_y_val.std(), adj_y_val.max())

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(np.sqrt(adj_y_val))

In [ ]:
# One time delete 0.01%
over_index = np.where(adj_y_val < std_pre_ave)[0]
print(over_index)
print(len(adj_y_val), len(over_index))
drop_rate = 0.02
drop_number = int(len(over_index) * drop_rate) if int(len(over_index) * drop_rate) > 1.0 else 1
print(drop_number)

adj_area = 0.0001

In [ ]:
%%time
for i in range(100):
    over_index = np.where(adj_y_val < std_pre_ave)[0]
    np.random.shuffle(over_index)
    drop_index = over_index[:drop_number]
    drop_number = int(drop_number * 0.95)
    
    adj_x_val = np.delete(adj_x_val, drop_index, 0)
    adj_y_val = np.delete(adj_y_val, drop_index)
    if adj_y_val.mean() + adj_area > std_pre_ave:
        break
print(f'Stop i:{i}')
print(adj_y_val.mean(), adj_y_val.std(), adj_y_val.max())

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(np.sqrt(adj_y_val))

In [ ]:
del x_train, y_train, x_val, y_val
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
n_models = []

In [ ]:
tpu = None
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except ValueError:
    # Enable XLA
    tf.config.optimizer.set_jit(enabled = "autoclustering")
    strategy = tf.distribute.get_strategy()
    
# Set Auto Tune
AUTOTUNE = tf.data.experimental.AUTOTUNE   

In [ ]:
BATCH_SIZE = 2048
SHUFFLE_BUFFER_SIZE = 100

In [ ]:
dataset_train = adj_x_train.reshape(adj_x_train.shape + (1,))
dataset_val = adj_x_val.reshape(adj_x_val.shape + (1,))
dataset_train = tf.data.Dataset.from_tensor_slices((dataset_train, adj_y_train))
dataset_val = tf.data.Dataset.from_tensor_slices((dataset_val, adj_y_val))

dataset_train = dataset_train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE)

In [ ]:
del adj_x_train, adj_y_train, adj_x_val, adj_y_val
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
dataset_train

In [ ]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(input_width, len(features), 1)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(32),
        tf.keras.layers.Dense(units=1)
    ])
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error',
                                                        mode='min', patience=3)

    model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.RootMeanSquaredError()])

    # val_performance['Dense'] = dense.evaluate(val_window.val)
    # performance['Dense'] = dense.evaluate(val_window.test, verbose=0)
    model.summary()

In [ ]:
model.fit(dataset_train, epochs=30,
    validation_data=dataset_val,
    callbacks=[early_stopping],
    batch_size=BATCH_SIZE,
)
n_models.append(model)

In [ ]:
del model, dataset_train, dataset_val
tf.keras.backend.clear_session()
gc.collect()

In [ ]:
# dataset_train = tf.data.Dataset.from_tensor_slices((adj_x_train, adj_y_train))
# dataset_val = tf.data.Dataset.from_tensor_slices((adj_x_val, adj_y_val))

# dataset_train = dataset_train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# dataset_val = dataset_val.batch(BATCH_SIZE)

In [ ]:
# model = tf.keras.Sequential([
# #     tf.keras.layers.ConvLSTM2D(64, (3,3),
# #         input_shape=(adj_x_train.shape[1], adj_x_train.shape[2], 1),
# #         padding="same",
# #         return_sequences=True,
# #         activation="relu",
# #     ),
# #     tf.keras.layers.ConvLSTM2D(64, (3,3), padding="same", return_sequences=True, activation="relu"),
# #     tf.keras.layers.GlobalAveragePooling2D(),
#     tf.keras.layers.LSTM(32, return_sequences=True),
#     tf.keras.layers.LSTM(32, return_sequences=True),
#     tf.keras.layers.Dense(1)
# ])
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_root_mean_squared_error',
#                                                     mode='min', patience=5)

# model.compile(loss=tf.losses.MeanSquaredError(),
#             optimizer=tf.optimizers.Adam(),
#             metrics=[tf.metrics.RootMeanSquaredError()])

In [ ]:
# model.fit(dataset_train, epochs=30,
#     validation_data=dataset_val,
#     callbacks=[early_stopping],
#     batch_size=BATCH_SIZE,
# )

In [ ]:
def flatten_x(x):
    flat_row = []
    for one_row in x:
        temp_flat_row = []
        for feature_set in one_row:
            for one_feature in feature_set:
                temp_flat_row.append(one_feature)
        flat_row.append(temp_flat_row)
        
    return flat_row

In [ ]:
# adj_x_train = flatten_x(adj_x_train)
# adj_x_val = flatten_x(adj_x_val)

In [ ]:
models = []

In [ ]:
# lgb_train = lgb.Dataset(adj_x_train, adj_y_train, free_raw_data=False)
# lgb_val = lgb.Dataset(adj_x_val, adj_y_val, reference=lgb_train, free_raw_data=False)


# lgbm_params = {
#     'objective': 'mse',
#     'metric': 'rmse',
#     "num_leaves": 500,
#     'is_unbalance':True,
#     'boosting':'gbdt',
#     "learning_rate": 0.01,
#     'num_boost_round': 10000,
#     'early_stopping_rounds':200
# }

# # # optimized by oputuna
# # lgbm_params = {
# #     'objective': 'mse',
# #     'metric' : 'rmse',
# #     "num_leaves": 966,
# #     "cat_smooth": 45.01680827234465,
# #     "min_child_samples": 27,
# #     "min_child_weight": 0.021144950289224463,
# #     "max_bin": 214,
# #     "learning_rate": 0.01,
# #     "subsample_for_bin": 300000,
# #     "min_data_in_bin": 7,
# #     "colsample_bytree": 0.8,
# #     "subsample": 0.6,
# #     "subsample_freq": 5,
# #     "n_estimators": 3000,
# # }


In [ ]:
# model = lgb.train(lgbm_params,
#                   lgb_train,
#                   valid_names=['train', 'valid'],
#                   valid_sets=[lgb_train, lgb_val],
#                   early_stopping_rounds=20,
#                   verbose_eval=100)
# models.append(model)

In [ ]:
# model = catboost.CatBoostRegressor(
#     iterations=700,
#     learning_rate=0.02,
#     depth=12,
#     eval_metric='RMSE',
#     random_seed = 23,
#     bagging_temperature = 0.2,
#     od_type='Iter',
#     metric_period = 75,
#     od_wait=100
# )

# model.fit(
#     adj_x_train,
#     adj_y_train, 
#     eval_set=(adj_x_val,adj_y_val),
# )
# models.append(model)

In [ ]:
# model = XGBRegressor(
#     max_depth=8,
#     n_estimators=1000,
#     min_child_weight=300, 
#     colsample_bytree=0.8, 
#     subsample=0.8, 
#     eta=0.3,    
#     seed=42,
# )
# model.fit(
#     adj_x_train,
#     adj_y_train, 
#     eval_metric="rmse", 
#     eval_set=[(adj_x_train, adj_y_train), (adj_x_val, adj_y_val)], 
#     verbose=True, 
#     early_stopping_rounds = 10)
# models.append(model)

In [ ]:
sub_x = train_window.generate_sub_x()

In [ ]:
# CNN
dataset_sub = np.array(sub_x)
dataset_sub = dataset_sub.reshape(dataset_sub.shape + (1,))
dataset_sub = tf.data.Dataset.from_tensor_slices(dataset_sub)
dataset_sub = dataset_sub.batch(BATCH_SIZE)
dataset_sub

In [ ]:
del features, transition, std_transition, shop_transition, item_transition
tf.keras.backend.clear_session()
gc.collect()

BATCH_SIZE = 256

In [ ]:
preds = n_models[0].predict(dataset_sub, batch_size=BATCH_SIZE)
# preds = np.zeros(len(sub_x))
# for model in n_models:
#     preds = preds + model.predict(dataset_sub, batch_size=BATCH_SIZE)

In [ ]:
# # Gradient boosting models
# sub_x = flatten_x(sub_x)

# preds = np.zeros(len(sub_x))
# for model in models:
#     preds = preds + model.predict(sub_x)
# preds = preds / len(models)

In [ ]:
# preds = model.predict(sub_x)
preds = preds * transition_max
preds = preds.clip(0.0, 20.0)

In [ ]:
print(preds.mean(), preds.std(), preds.max())

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(preds,bins=50)

In [ ]:
plt.figure(figsize=(12,6))
plt.hist(preds[(preds>0.0) & (preds<1.0)],bins=50)

In [ ]:
# stop_unit = 0.01

# adj_area = preds.mean() * 2

# min_unit = 0.01
# if (preds.mean() > pre_ave):
#     min_unit *= -1
# min_rate = 1 + min_unit

# print(adj_area)
# print(min_rate)

In [ ]:
# adj_preds = preds.copy()
# for i in range(1000):
#     adj_preds *= min_rate
# #     adj_preds[adj_preds < adj_area] = (adj_preds[adj_preds < adj_area] - pre_ave) * min_rate + pre_ave
#     if (adj_preds.mean() - stop_unit < pre_ave) and (adj_preds.mean() + stop_unit > pre_ave):
#         break
# print(f'Stop i:{i}, mean:{adj_preds.mean()}')

In [ ]:
# plt.figure(figsize=(12,6))
# plt.hist(adj_preds,bins=50)

In [ ]:
sample['item_cnt_month'] = preds
sample.to_csv('submission.csv', index=False)

In [ ]:
sample.head(50)

In [ ]:
plt.hist(sample['item_cnt_month'],bins=50)